In [1]:
# 2025-10-19 run it from "python 3" (ipykernel) on gl-base-env-j4-ces with 1 T4 gpu


# preparing data

## copy data from gcs bucket 
merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data

In [ ]:
# gsutil ls -r gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data/
# gsutil -m cp -r gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data/* /home/jupyter/mluser/git/cube/data/dataspace/polyhaven_tiny/

# (base) jupyter@gl-base-env-j4-ces:~/mluser/git/cube/data/dataspace/polyhaven_tiny$ ls -lh
# total 1.6G
# -rw-r--r-- 1 jupyter jupyter 1.5G Oct 22 00:03 cubediff_train.tar
# -rw-r--r-- 1 jupyter jupyter 111M Oct 22 00:02 cubediff_val.tar
# drwxr-xr-x 2 jupyter jupyter  36K Oct 22 00:02 latents
# drwxr-xr-x 6 jupyter jupyter 4.0K Oct 22 00:03 raw


# set up env

In [11]:
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/cube"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"
os.environ["ACCELERATE_CONFIG_FILE"]="/home/jupyter/mluser/git/cube/cl/config/accelerate_config.yaml"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"

## install packages

In [2]:
# before installation, check what installed
# (base) jupyter@gl-base-env-j4-ces:~/mluser/git/cube/data/dataspace/polyhaven_tiny$ pip list | grep "torch\|diff\|peft\|hug\|acc"
# torch                                    2.8.0
# torch-model-archiver                     0.12.0
# torchserve                               0.12.0

In [2]:
# # 2015-10-21
# # Install compatible versions without PEFT
# !pip install diffusers==0.21.4 huggingface-hub==0.16.4 -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

# # Upgrade to versions that work together
# # !pip install huggingface-hub==0.21.4 accelerate==1.0.1 -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple
# !pip install accelerate==1.0.1 -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

# # 2025-10-21 Upgrade to latest compatible versions
# # !pip install huggingface-hub==0.25.1 transformers==4.35.2 diffusers==0.25.1 -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple
# !pip install huggingface-hub==0.25.1 transformers==4.35.2 diffusers==0.25.1 -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

Looking in indexes: https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple
  Using cached https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/packages/packages/2e/ed/58a13f88dfcdd1bcfeabf44d4c9861979551339348e579fd6559c64b12e0/diffusers-0.21.4-py3-none-any.whl (1.5 MB)
  Using cached https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/packages/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.25.1
    Uninstalling huggingface-hub-0.25.1:
      Successfully uninstalled huggingface-hub-0.25.1
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.25.1━━━━━━━━━━━━━━━━━ 1/2 [diffusers]
    Uninstalling diffusers-0.25.1:━━━━━━━━━━━━━━━━━━━ 1/2 [diffusers]
      Successfully uninstalled diffusers-0.25.1━━━━━━━━━━━━━━━ 1/2 [diffusers]
  

In [56]:
!ls -lha /home/jupyter/.local/

total 12K
drwxr-xr-x  3 jupyter jupyter 4.0K Jul 31 19:52 .
drwxr-xr-x 16 jupyter jupyter 4.0K Oct 21 00:13 ..
drwxr-xr-x  3 jupyter jupyter 4.0K Jul 31 19:52 share


In [57]:
!ls -lha /home/jupyter/.local/share

total 12K
drwxr-xr-x 3 jupyter jupyter 4.0K Jul 31 19:52 .
drwxr-xr-x 3 jupyter jupyter 4.0K Jul 31 19:52 ..
drwxr-xr-x 3 jupyter jupyter 4.0K Nov 18 00:12 jupyter


In [59]:
!ls -lha /home/jupyter/.local/share/jupyter

total 204K
drwxr-xr-x 3 jupyter jupyter 4.0K Nov 18 00:12 .
drwxr-xr-x 3 jupyter jupyter 4.0K Jul 31 19:52 ..
-rw-r--r-- 1 jupyter jupyter 184K Nov 18 00:12 nbsignatures.db
-rw------- 1 jupyter jupyter 1.4K Jul 31 19:59 notebook_secret
drwx-----T 2 jupyter jupyter 4.0K Nov 17 19:45 runtime


In [60]:
!ls -lha /home/jupyter/.local/share/jupyter/runtime

total 52K
drwx-----T 2 jupyter jupyter 4.0K Nov 17 19:45 .
drwxr-xr-x 3 jupyter jupyter 4.0K Nov 18 00:12 ..
-rw-r--r-- 1 jupyter jupyter  398 Nov 17 18:16 jpserver-4965-open.html
-rw------- 1 jupyter jupyter  244 Nov 17 18:16 jpserver-4965.json
-rw------- 1 jupyter jupyter   45 Jul 31 19:52 jupyter_cookie_secret
-rw------- 1 jupyter jupyter  276 Nov 17 18:17 kernel-0613fb14-8c80-4a8e-8296-dcdac95d9a53.json
-rw------- 1 jupyter jupyter  276 Nov 17 18:17 kernel-3ce5dde7-7a6b-4a29-8a87-dc403b116e7c.json
-rw------- 1 jupyter jupyter  276 Nov 17 18:17 kernel-63d94742-2239-4822-9508-1dbcdd4533d3.json
-rw------- 1 jupyter jupyter  276 Nov 17 18:17 kernel-7d500bd1-c777-435f-a4fb-e817db061135.json
-rw------- 1 jupyter jupyter  276 Nov 17 19:45 kernel-b23b5a14-d644-458a-b961-eeb70433b3b2.json
-rw------- 1 jupyter jupyter  276 Nov 17 18:17 kernel-dbce4daf-0dde-4ba8-ac4a-f0a9b514dfff.json
-rw------- 1 jupyter jupyter  276 Nov 17 18:48 kernel-e12f1b1a-cbd8-49b4-afdc-b36ed198c4e7.json
-rw------- 1 

In [48]:
!which pip

/opt/conda/bin/pip


In [50]:
!which python

/opt/conda/bin/python


In [64]:
!ls -lha /opt/conda/lib/python3.10/site-packages/transformers*


/opt/conda/lib/python3.10/site-packages/transformers:
total 2.6M
drwxr-xr-x  15 jupyter jupyter 4.0K Nov 18 00:01 .
drwxrwxrwx 704 root    root     36K Nov 18 00:01 ..
-rw-r--r--   1 jupyter jupyter 291K Nov 18 00:01 __init__.py
drwxr-xr-x   2 jupyter jupyter 4.0K Nov 18 00:01 __pycache__
-rw-r--r--   1 jupyter jupyter 8.6K Nov 18 00:01 activations.py
-rw-r--r--   1 jupyter jupyter 4.3K Nov 18 00:01 activations_tf.py
-rw-r--r--   1 jupyter jupyter  32K Nov 18 00:01 audio_utils.py
drwxr-xr-x   3 jupyter jupyter 4.0K Nov 18 00:01 benchmark
drwxr-xr-x   3 jupyter jupyter 4.0K Nov 18 00:01 commands
-rw-r--r--   1 jupyter jupyter  54K Nov 18 00:01 configuration_utils.py
-rw-r--r--   1 jupyter jupyter  21K Nov 18 00:01 convert_graph_to_onnx.py
-rw-r--r--   1 jupyter jupyter  17K Nov 18 00:01 convert_pytorch_checkpoint_to_tf2.py
-rw-r--r--   1 jupyter jupyter  52K Nov 18 00:01 convert_slow_tokenizer.py
-rw-r--r--   1 jupyter jupyter 4.9K Nov 18 00:01 convert_slow_tokenizers_checkpoints_to_fas

In [65]:
!sudo rm -rf /opt/conda/lib/python3.10/site-packages/transformers
!sudo rm -rf /opt/conda/lib/python3.10/site-packages/transformers-*


In [66]:
!sudo rm -rf /opt/conda/lib/python3.10/site-packages/accelerate
!sudo rm -rf /opt/conda/lib/python3.10/site-packages/accelerate-*


In [69]:
!pip install transformers==4.35.2 \
    diffusers==0.25.1 huggingface_hub==0.25.1 \
    accelerate==0.22.0 -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple


Looking in indexes: https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple
  Using cached https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/packages/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl (7.9 MB)
  Using cached https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/packages/packages/4d/a7/05c67003d659a0035f2b3a8cf389c1d9645865aee84a73ce99ddab16682f/accelerate-0.22.0-py3-none-any.whl (251 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [accelerate]2 [accelerate]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.35.2 which is incompatible.


In [70]:
import transformers, inspect
print(transformers.__version__)
print(inspect.getfile(transformers))


4.35.2
/opt/conda/lib/python3.10/site-packages/transformers/__init__.py


In [72]:
!ls -lh /opt/conda/lib/python3.10/site-packages/transformers/integrations/


total 152K
-rw-r--r-- 1 jupyter jupyter 4.3K Nov 18 00:19 __init__.py
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 00:19 __pycache__
-rw-r--r-- 1 jupyter jupyter 4.5K Nov 18 00:19 awq.py
-rw-r--r-- 1 jupyter jupyter  14K Nov 18 00:19 bitsandbytes.py
-rw-r--r-- 1 jupyter jupyter  18K Nov 18 00:19 deepspeed.py
-rw-r--r-- 1 jupyter jupyter  71K Nov 18 00:19 integration_utils.py
-rw-r--r-- 1 jupyter jupyter  22K Nov 18 00:19 peft.py


In [13]:
# 2025-10-25, 2025-7-7 installed compatible diffusers huggingface_hub transformers to support torch.compile
# !pip install torch==2.1.2 torchvision==0.16.2  \
#      opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
#      xformers requests pillow openexr opencv-python-headless wandb \
#      bitsandbytes webdataset timm accelerate deepspeed imageio \
#      diffusers==0.25.1 huggingface_hub==0.25.1 transformers==4.35.2 \
#      -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

!pip install torch==2.1.2 torchvision==0.16.2 \
    opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
    xformers requests pillow openexr opencv-python-headless wandb \
    bitsandbytes webdataset timm imageio \
    diffusers==0.25.1 huggingface_hub==0.25.1 transformers==4.35.2 \
    accelerate==0.22.0 deepspeed \
    -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple


Looking in indexes: https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple


In [67]:
!ls /opt/conda/lib/python3.10/site-packages/transformers/integrations/


ls: cannot access '/opt/conda/lib/python3.10/site-packages/transformers/integrations/': No such file or directory


In [49]:
import transformers, inspect
print("Transformers version:", transformers.__version__)
print("Loaded from:", inspect.getfile(transformers))


Transformers version: 4.35.2
Loaded from: /opt/conda/lib/python3.10/site-packages/transformers/__init__.py


In [2]:
import OpenEXR

In [14]:
!pip show OpenEXR

Name: OpenEXR
Version: 3.4.2
Summary: Python bindings for the OpenEXR image file format
Home-page: https://openexr.com
Author: 
Author-email: Contributors to the OpenEXR project <info@openexr.com>
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy
Required-by: 


## check packages versions

In [73]:
!pip list | grep "diff\|hug\|acc\|trans\|torch\|peft"

accelerate                               0.22.0
diffusers                                0.25.1
huggingface-hub                          0.25.1
sentence-transformers                    5.1.2
torch                                    2.1.2
torch-model-archiver                     0.12.0
torchserve                               0.12.0
torchvision                              0.16.2
transformers                             4.35.2


# config shm

In [16]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             59G     0   59G   0% /dev
tmpfs            12G  768K   12G   1% /run
/dev/sda1       148G   45G   98G  32% /
tmpfs            64G   20K   64G   1% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
/dev/sda15      124M   11M  114M   9% /boot/efi
/dev/sdb        147G   40G  108G  27% /home/jupyter


In [17]:
# allocate more memory for shm
!sudo mount -o remount,size=64G /dev/shm

In [18]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             59G     0   59G   0% /dev
tmpfs            12G  768K   12G   1% /run
/dev/sda1       148G   45G   98G  32% /
tmpfs            64G   20K   64G   1% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
/dev/sda15      124M   11M  114M   9% /boot/efi
/dev/sdb        147G   40G  108G  27% /home/jupyter


# copied HF/hub and torch.hub from gcs to this VM

In [ ]:
# on target VM:
# mkdir -p ~/.cache/huggingface/hub
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/* \
#   ~/.cache/huggingface/hub/


In [23]:
# !mkdir -p ~/.cache/huggingface/hub

In [21]:
!gsutil -m cp -r \
   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/* \
   ~/.cache/huggingface/hub/


Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/version.txt...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/version_diffusers_cache.txt...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/.locks/models--openai--clip-vit-base-patch32/182766ce89b439768edadda342519f33802f5364.lock...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/.locks/models--openai--clip-vit-base-patch32/4fdaf6842dd5a725b940c92b6b692490ce59d548.lock...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/.locks/models--openai--clip-vit-base-patch32/564c0ebd5ce29c4ee4864004aee693deadd3128c.lock...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/.locks/models--openai--clip-vit-base-patch32/9bfb42aa97dcd61e89f279ccaee988bccb4fabae.lock...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/.loc

In [19]:
!ls -lh ~/.cache/huggingface/hub

total 20K
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 17 19:40 clip-vit-base-patch32
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--openai--clip-vit-large-patch14
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--runwayml--stable-diffusion-v1-5
-rw-r--r-- 1 jupyter jupyter    1 Nov 17 19:40 version.txt
-rw-r--r-- 1 jupyter jupyter    1 Nov 17 19:40 version_diffusers_cache.txt


In [20]:
!rm -rf ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32

In [21]:
# on target VM:
# 2025-11-17 this is important to copy the new "models--openai--clip-vit-base-patch32/"
# !gsutil -m cp -r gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/* \
#   ~/.cache/huggingface/hub/


Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/182766ce89b439768edadda342519f33802f5364...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/4fdaf6842dd5a725b940c92b6b692490ce59d548...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/564c0ebd5ce29c4ee4864004aee693deadd3128c...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/5a12a1eb250987a4eee0e3e7d7338c4b22724be1...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/blobs/99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d...
Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/models--openai--clip-vit-base-patch32/

In [22]:
!ls -lh ~/.cache/huggingface/hub

total 24K
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 17 19:40 clip-vit-base-patch32
drwxr-xr-x 5 jupyter jupyter 4.0K Nov 18 00:03 models--openai--clip-vit-base-patch32
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--openai--clip-vit-large-patch14
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--runwayml--stable-diffusion-v1-5
-rw-r--r-- 1 jupyter jupyter    1 Nov 17 19:40 version.txt
-rw-r--r-- 1 jupyter jupyter    1 Nov 17 19:40 version_diffusers_cache.txt


In [ ]:
# 2025-11-17 no need to run
# on target VM:
# mkdir -p ~/.cache/torch/hub
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/* \
#   ~/.cache/torch/hub/


In [26]:
# !mkdir -p ~/.cache/torch/hub

In [28]:
# 2025-11-17 no need to run
# !gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/* \
#   ~/.cache/torch/hub/


Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/checkpoints/vgg16-397923af.pth...
\ [1/1 files][527.8 MiB/527.8 MiB] 100% Done                                    
Operation completed over 1 objects/527.8 MiB.                                    


In [23]:
!ls -lh ~/.cache/torch/hub/

total 4.0K
drwxr-xr-x 2 jupyter jupyter 4.0K Oct 25 06:27 checkpoints


In [24]:
!ls -lh ~/.cache/torch/hub/checkpoints

total 528M
-rw-r--r-- 1 jupyter jupyter 528M Oct 25 06:27 vgg16-397923af.pth


# check models

In [74]:
!ls -lh ~/.cache/huggingface/hub/

total 24K
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 17 19:40 clip-vit-base-patch32
drwxr-xr-x 5 jupyter jupyter 4.0K Nov 18 00:03 models--openai--clip-vit-base-patch32
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--openai--clip-vit-large-patch14
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--runwayml--stable-diffusion-v1-5
-rw-r--r-- 1 jupyter jupyter    1 Nov 17 19:40 version.txt
-rw-r--r-- 1 jupyter jupyter    1 Nov 17 19:40 version_diffusers_cache.txt


In [75]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/:
total 12K
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 00:03 blobs
drwxr-xr-x 3 jupyter jupyter 4.0K Nov 18 00:03 refs
drwxr-xr-x 4 jupyter jupyter 4.0K Nov 18 00:03 snapshots

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs:
total 1.2G
-rw-r--r-- 1 jupyter jupyter 843K Nov 18 00:03 182766ce89b439768edadda342519f33802f5364
-rw-r--r-- 1 jupyter jupyter  592 Nov 18 00:03 4fdaf6842dd5a725b940c92b6b692490ce59d548
-rw-r--r-- 1 jupyter jupyter 2.2M Nov 18 00:03 564c0ebd5ce29c4ee4864004aee693deadd3128c
-rw-r--r-- 1 jupyter jupyter  316 Nov 18 00:03 5a12a1eb250987a4eee0e3e7d7338c4b22724be1
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 00:03 99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d
-rw-r--r-- 1 jupyter jupyter  389 Nov 18 00:03 9bfb42aa97dcd61e89f279ccaee988bccb4fabae
-rw-r--r-- 1 jupyter jupyter 4.1K Nov 18 00:03 a2a88b96561196777ca173b15309ea859f4d2ce0
-rw-r--r-- 1 jupy

In [76]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs 

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs:
total 1.2G
-rw-r--r-- 1 jupyter jupyter 843K Nov 18 00:03 182766ce89b439768edadda342519f33802f5364
-rw-r--r-- 1 jupyter jupyter  592 Nov 18 00:03 4fdaf6842dd5a725b940c92b6b692490ce59d548
-rw-r--r-- 1 jupyter jupyter 2.2M Nov 18 00:03 564c0ebd5ce29c4ee4864004aee693deadd3128c
-rw-r--r-- 1 jupyter jupyter  316 Nov 18 00:03 5a12a1eb250987a4eee0e3e7d7338c4b22724be1
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 00:03 99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d
-rw-r--r-- 1 jupyter jupyter  389 Nov 18 00:03 9bfb42aa97dcd61e89f279ccaee988bccb4fabae
-rw-r--r-- 1 jupyter jupyter 4.1K Nov 18 00:03 a2a88b96561196777ca173b15309ea859f4d2ce0
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 00:03 a63082132ba4f97a80bea76823f544493bffa8082296d62d71581a4feff1576f
-rw-r--r-- 1 jupyter jupyter 513K Nov 18 00:03 bbfec752c9a675946c6dce106def6f35c882dcc2


In [77]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/182766ce89b439768edadda342519f33802f5364

-rw-r--r-- 1 jupyter jupyter 843K Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/182766ce89b439768edadda342519f33802f5364


In [78]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/4fdaf6842dd5a725b940c92b6b692490ce59d548

-rw-r--r-- 1 jupyter jupyter 592 Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/4fdaf6842dd5a725b940c92b6b692490ce59d548


In [79]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/564c0ebd5ce29c4ee4864004aee693deadd3128c

-rw-r--r-- 1 jupyter jupyter 2.2M Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/564c0ebd5ce29c4ee4864004aee693deadd3128c


In [80]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/9bfb42aa97dcd61e89f279ccaee988bccb4fabae

-rw-r--r-- 1 jupyter jupyter 389 Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/9bfb42aa97dcd61e89f279ccaee988bccb4fabae


In [81]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/a2a88b96561196777ca173b15309ea859f4d2ce0

-rw-r--r-- 1 jupyter jupyter 4.1K Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/a2a88b96561196777ca173b15309ea859f4d2ce0


In [82]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/bbfec752c9a675946c6dce106def6f35c882dcc2

-rw-r--r-- 1 jupyter jupyter 513K Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/bbfec752c9a675946c6dce106def6f35c882dcc2


In [83]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/

total 8.0K
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 00:03 3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 00:03 c237dc49a33fc61debc9276459120b7eac67e7ef


In [84]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268

total 581M
-rw-r--r-- 1 jupyter jupyter 4.1K Nov 18 00:03 config.json
-rw-r--r-- 1 jupyter jupyter 513K Nov 18 00:03 merges.txt
-rw-r--r-- 1 jupyter jupyter  316 Nov 18 00:03 preprocessor_config.json
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 00:03 pytorch_model.bin
-rw-r--r-- 1 jupyter jupyter  389 Nov 18 00:03 special_tokens_map.json
-rw-r--r-- 1 jupyter jupyter 2.2M Nov 18 00:03 tokenizer.json
-rw-r--r-- 1 jupyter jupyter  592 Nov 18 00:03 tokenizer_config.json
-rw-r--r-- 1 jupyter jupyter 843K Nov 18 00:03 vocab.json


In [85]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/182766ce89b439768edadda342519f33802f5364

ls: cannot access '/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/182766ce89b439768edadda342519f33802f5364': No such file or directory


In [86]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/refs/

total 8.0K
-rw-r--r-- 1 jupyter jupyter   40 Nov 18 00:03 main
drwxr-xr-x 3 jupyter jupyter 4.0K Nov 18 00:03 refs


In [87]:
!ls -lh ~/.cache/torch/hub/checkpoints

total 528M
-rw-r--r-- 1 jupyter jupyter 528M Oct 25 06:27 vgg16-397923af.pth


## test they can be loaded from local cache

In [88]:
!ls -lh  ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268


total 581M
-rw-r--r-- 1 jupyter jupyter 4.1K Nov 18 00:03 config.json
-rw-r--r-- 1 jupyter jupyter 513K Nov 18 00:03 merges.txt
-rw-r--r-- 1 jupyter jupyter  316 Nov 18 00:03 preprocessor_config.json
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 00:03 pytorch_model.bin
-rw-r--r-- 1 jupyter jupyter  389 Nov 18 00:03 special_tokens_map.json
-rw-r--r-- 1 jupyter jupyter 2.2M Nov 18 00:03 tokenizer.json
-rw-r--r-- 1 jupyter jupyter  592 Nov 18 00:03 tokenizer_config.json
-rw-r--r-- 1 jupyter jupyter 843K Nov 18 00:03 vocab.json


In [89]:
!pip list | grep "torch\|trans"

sentence-transformers                    5.1.2
torch                                    2.1.2
torch-model-archiver                     0.12.0
torchserve                               0.12.0
torchvision                              0.16.2
transformers                             4.35.2


In [1]:
from transformers import CLIPModel, CLIPProcessor

# This should now work without network calls
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=True, local_files_only=False) # use_safetensors=True, caused the error of no safetensors for the downloaded model.
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", local_files_only=True)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=False, local_files_only=True) # worked for the model downloaded on 2025-10-29 in "bbyus-trn-amlo-yieldoptimz-p01"
                                                                                                                # details at mluser/git/cube/training/notebooks/v2/download_copy_vit.ipynb
                                                                                                                # but the same vit model could be loaded well with "use_safetensors=True" 
                                                                                                                # in "bbyus-trn-amlo-yieldoptimz-p01"
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", local_files_only=True)

/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/conda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/opt/conda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/opt/conda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/opt/conda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
model.name_or_path

'openai/clip-vit-base-patch32'

# run model 

In [12]:
# 2025-11-17 Jupyter’s os.environ[...] = ... affects only the notebook kernel, not the subprocesses.
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/cube/"

import sys
sys.path.insert(0, "/home/jupyter/mluser/git/cube")

os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"
os.environ["ACCELERATE_CONFIG_FILE"]="/home/jupyter/mluser/git/cube/config/accelerate_config.yaml"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] =  "max_split_size_mb:64,expandable_segments:True"

In [13]:
ACCELERATE_CONFIG_FILE="/home/jupyter/mluser/git/cube/config/accelerate_config.yaml"

In [14]:
# !which accelerate
from training.trainer import CubeDiffTrainer 

In [15]:
!accelerate launch --config_file $ACCELERATE_CONFIG_FILE train_cubediff.py --cfg ../../../config/tiny_fullrank.yaml > cubediff_log.txt

Loading pipeline components...:  43%|█████▌       | 3/7 [00:01<00:02,  1.79it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...: 100%|█████████████| 7/7 [00:02<00:00,  2.39it/s]
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Traceback (most recent call last):
  File "/home/jupyter/mluser/git/cube/training/notebooks/v2/train_cubediff.py", line 131, in <module>

In [96]:
!ls -lha /opt/conda/lib/python3.10/site-packages/transformers*


/opt/conda/lib/python3.10/site-packages/transformers:
total 2.6M
drwxr-xr-x  15 jupyter jupyter 4.0K Nov 18 00:19 .
drwxrwxrwx 704 root    root     36K Nov 18 00:19 ..
-rw-r--r--   1 jupyter jupyter 291K Nov 18 00:19 __init__.py
drwxr-xr-x   2 jupyter jupyter 4.0K Nov 18 00:19 __pycache__
-rw-r--r--   1 jupyter jupyter 8.6K Nov 18 00:19 activations.py
-rw-r--r--   1 jupyter jupyter 4.3K Nov 18 00:19 activations_tf.py
-rw-r--r--   1 jupyter jupyter  32K Nov 18 00:19 audio_utils.py
drwxr-xr-x   3 jupyter jupyter 4.0K Nov 18 00:19 benchmark
drwxr-xr-x   3 jupyter jupyter 4.0K Nov 18 00:19 commands
-rw-r--r--   1 jupyter jupyter  54K Nov 18 00:19 configuration_utils.py
-rw-r--r--   1 jupyter jupyter  21K Nov 18 00:19 convert_graph_to_onnx.py
-rw-r--r--   1 jupyter jupyter  17K Nov 18 00:19 convert_pytorch_checkpoint_to_tf2.py
-rw-r--r--   1 jupyter jupyter  52K Nov 18 00:19 convert_slow_tokenizer.py
-rw-r--r--   1 jupyter jupyter 4.9K Nov 18 00:19 convert_slow_tokenizers_checkpoints_to_fas

In [94]:
!pip list | grep "trans\|diff\|acc\|hug\|torch"

accelerate                               0.22.0
diffusers                                0.25.1
huggingface-hub                          0.25.1
sentence-transformers                    5.1.2
torch                                    2.1.2
torch-model-archiver                     0.12.0
torchserve                               0.12.0
torchvision                              0.16.2
transformers                             4.35.2


In [97]:
!rm -rf /opt/conda/lib/python3.10/site-packages/transformers
!rm -rf /opt/conda/lib/python3.10/site-packages/transformers-*


In [98]:
!rm -rf /opt/conda/lib/python3.10/site-packages/accelerate
!rm -rf /opt/conda/lib/python3.10/site-packages/accelerate-*


In [101]:
!pip install transformers==4.35.2 \
    diffusers==0.25.1 huggingface_hub==0.25.1 \
    accelerate==0.22.0 \
    -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

Looking in indexes: https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple
  Using cached https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/packages/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl (7.9 MB)
  Using cached https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/packages/packages/4d/a7/05c67003d659a0035f2b3a8cf389c1d9645865aee84a73ce99ddab16682f/accelerate-0.22.0-py3-none-any.whl (251 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [accelerate]s]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.35.2 which is incompatible.


In [102]:
!pip list | grep "torch\|trans\|hug\|acc"

accelerate                               0.22.0
huggingface-hub                          0.25.1
sentence-transformers                    5.1.2
torch                                    2.1.2
torch-model-archiver                     0.12.0
torchserve                               0.12.0
torchvision                              0.16.2
transformers                             4.35.2


In [95]:
!ls -lh /opt/conda/lib/python3.10/site-packages/transformers/integrations

total 152K
-rw-r--r-- 1 jupyter jupyter 4.3K Nov 18 00:19 __init__.py
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 00:19 __pycache__
-rw-r--r-- 1 jupyter jupyter 4.5K Nov 18 00:19 awq.py
-rw-r--r-- 1 jupyter jupyter  14K Nov 18 00:19 bitsandbytes.py
-rw-r--r-- 1 jupyter jupyter  18K Nov 18 00:19 deepspeed.py
-rw-r--r-- 1 jupyter jupyter  71K Nov 18 00:19 integration_utils.py
-rw-r--r-- 1 jupyter jupyter  22K Nov 18 00:19 peft.py


In [19]:
!ls -lh /home/jupyter/.local/share/jupyter/

total 192K
-rw-r--r-- 1 jupyter jupyter 180K Nov 17 23:56 nbsignatures.db
-rw------- 1 jupyter jupyter 1.4K Jul 31 19:59 notebook_secret
drwx-----T 2 jupyter jupyter 4.0K Nov 17 19:45 runtime


In [21]:
!pip uninstall -y transformers

In [109]:
# Confirm the old path is gone
import transformers, inspect
import sys
print("Transformers version:", transformers.__version__)
print("Loaded from:", inspect.getfile(transformers))

Transformers version: 4.35.2
Loaded from: /opt/conda/lib/python3.10/site-packages/transformers/__init__.py


In [111]:
import accelerate, inspect
print(accelerate.__version__)
print(inspect.getfile(accelerate))


0.22.0
/opt/conda/lib/python3.10/site-packages/accelerate/__init__.py


In [24]:
!ls /opt/conda/lib/python3.10/site-packages/transformers

ls: cannot access '/opt/conda/lib/python3.10/site-packages/transformers': No such file or directory
